In [ ]:
import numpy as np
import pandas as pd
import csv
import itertools
import math
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pathlib
%matplotlib inline
from collections import Counter
from scipy import interpolate
from math import pi
from matplotlib import pyplot
from math import pi
from numpy import nan
from collections import Counter
import matplotlib.pyplot as plt

from collections import Counter
from scipy import interpolate
from scipy.stats import norm
from scipy.stats import lognorm
from scipy.stats import skewnorm
from scipy.stats import rice
from scipy.stats import gumbel_r
from scipy.stats import genextreme
from scipy.stats import genpareto
from scipy.stats import kstest
from scipy.stats import ks_2samp
from math import pi
from matplotlib import pyplot
from matplotlib.pyplot import figure
from math import pi
from numpy import nan
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import jensenshannon
from numpy import asarray

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from datetime import date, timedelta # Date Functions
from sklearn.metrics import mean_absolute_error, mean_squared_error # For measuring model performance / errors
from sklearn.preprocessing import MinMaxScaler #to normalize the price data 
from tensorflow.keras.models import Sequential # Deep learning library, used for neural networks
from tensorflow.keras.layers import LSTM, Dense, Conv2D, Conv1D, MaxPooling2D # Deep learning classes for recurrent and regular densely-connected layers
import tensorflow as tf
import seaborn as sns
sns.set_style('white', { 'axes.spines.right': False, 'axes.spines.top': False})
# check the tensorflow version and the number of available GPUs
print('Tensorflow Version: ' + tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))
from keras.callbacks import EarlyStopping

In [ ]:
# data_ADT is a one dimetional vehicle yearly volume data. 
# It one dimetional data with only one colume of timestamp and another column of vehicle volume
# First step is to make timestamp column as Index column

data_ADT = data[['year1', 'totadt1']].astype(int)
data_ADT = data_ADT.set_index('year1')
print(data_ADT.head())
data_ADT.plot()
plt.show()
############################################################## LSTM

#### Feature Selection - Only Close Data
train_df = data_ADT.filter(['totadt1'])
# train_df = df
# print(train_df)

from sklearn.utils import shuffle
# train_df = shuffle(train_df)
data_unscaled = train_df.values

# Get the number of rows to train the model on 80% of the data 
train_data_length = math.ceil(len(data_unscaled) * 0.7)

# Transform features by scaling each feature to a range between 0 and 1
mmscaler = MinMaxScaler(feature_range=(0, 1))
np_data = mmscaler.fit_transform(data_unscaled)
# print(np_data)

# Set the sequence length - this is the timeframe used to make a single prediction

sequence_length = 12

# Prediction Index
index_weight = train_df.columns.get_loc('totadt1')
print('Prediction Index: ', index_weight)
# Split the training data into train and train data sets
# As a first step, we get the number of rows to train the model on 80% of the data 
train_data_len = math.ceil(np_data.shape[0] * 0.7)

# Create the training and test data
train_data = np_data[0:train_data_len, :]
test_data = np_data[train_data_len - sequence_length:, :]

# The RNN needs data with the format of [samples, time steps, features]
# Here, we create N samples, sequence_length time steps per sample, and 6 features
def partition_dataset(sequence_length, train_df):
    x, y = [], []
    data_len = train_df.shape[0]
    for ij in range(sequence_length, data_len):
        x.append(train_df[ij-sequence_length:ij,:]) #contains sequence_length values 0-sequence_length * columsn
        y.append(train_df[ij, index_weight]) #contains the prediction values for validation (3rd column = Close),  
                                             #for single-step prediction

    # Convert the x and y to numpy arrays
    x = np.array(x)
    y = np.array(y)
    return x, y
from sklearn.model_selection import train_test_split
# Generate training data and test data
x_train, y_train = partition_dataset(sequence_length, train_data)
# print(x_train)
x_test, y_test = partition_dataset(sequence_length, test_data)

# Print the shapes: the result is: (rows, training_sequence, features) | (prediction value, )
print('X Train Shape: ', x_train.shape, '| Y Train Shape: ',y_train.shape)
print('X Test Shape: ', x_test.shape, '| Y Test Shape: ', y_test.shape)

# Validate that the prediction value and the input match up
# The last gross weight of the second input sample should equal the first prediction of gross weight value
print(x_test[1][sequence_length-1][index_weight])
print(y_test[0])

# Configure the neural network model

accuracy = []

epoch = 300
batch_size = 8
neurons = sequence_length     # Model with sequence_length Neurons 
model = Sequential()
model.add(LSTM(neurons, return_sequences=True, dropout=0.0, recurrent_dropout= 0, input_shape=(x_train.shape[1], 1))) 
model.add(LSTM(neurons, return_sequences=True, dropout=0.0, recurrent_dropout= 0))
model.add(LSTM(neurons, return_sequences=False, dropout=0.0, recurrent_dropout= 0))
model.add(Dense(25, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)
# Training the model
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epoch, 
                    validation_split = 0.2, shuffle=True, verbose= 0, callbacks=[es])
epoch_es = len(history.history['val_loss'])
print('Early Stop Epoch: ', epoch_es)
# Get the predicted values
y_pred_scaled = model.predict(x_test)
y_pred = mmscaler.inverse_transform(y_pred_scaled)
y_test_unscaled = mmscaler.inverse_transform(y_test.reshape(-1, 1))

# Mean Absolute Error (MAE)
MAE = mean_absolute_error(y_test_unscaled, y_pred)
print(f'Median Absolute Error (MAE): {np.round(MAE, 2)}')

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print(f'Mean Absolute Percentage Error (MAPE): {np.round(MAPE, 2)} %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled)) ) * 100
print(f'Median Absolute Percentage Error (MDAPE): {np.round(MDAPE, 2)} %')

# The date from which on the date is displayed
# display_start_date = "2018-01-01" 

# Add the difference between the valid and predicted weights
train = pd.DataFrame(train_df[:train_data_length + 1]).rename(columns={'totadt1': 'x_train'})
valid = pd.DataFrame(train_df[train_data_length:]).rename(columns={'totadt1': 'y_test'})
# print(valid)
valid.insert(1, "y_pred", y_pred, True)
valid.insert(1, "residuals", valid["y_pred"] - valid["y_test"], True)
df_union = pd.concat([train, valid])
# print(df_union)
rmse = np.sqrt(mean_squared_error(valid["y_pred"], valid["y_test"]))
print('RMSE:', str(rmse))


############### LSTM outcomes visualizations
# Zoom in to a closer timeframe
df_union_zoom = df_union
#         print(df_union_zoom)

# Create the lineplot
fig, ax = plt.subplots(figsize=(16, 5), sharex=True)
# plt.title("Weight Prediction vs Actual Weights (kips)(Epoch "+str(epoch_es)+")(Site ID "+str(SITE_ID)+")("+str(BOUND[i])+")", fontsize=14)
plt.ylabel('ADT')
plt.xlabel('Year')
sns.lineplot(data=df_union_zoom["x_train"], linewidth=1.0, linestyle = 'solid', color = 'gray')
sns.lineplot(data=df_union_zoom["y_test"], linewidth=1.0, linestyle = 'solid', color = 'black')
sns.lineplot(data=df_union_zoom["y_pred"], linewidth=1.0, linestyle = 'dashed', color = 'black')
sns.lineplot(data=df_union_zoom["residuals"], linewidth=1.0, linestyle = 'dotted', color = 'black')
plt.legend(["Training Data", "Test Data", "Test Prediction", "Residuals"], loc="best")
# plt.fill_between(data_ADT.index, under_line, over_line, color='black', alpha=.1)
plt.grid(axis = 'x')
# plt.savefig(new_path +'\\' + new_folder +'\\'+str(BOUND[i])+'~Test_Train_Residual plot (Site ID '+str(SITE_ID)+').png', dpi=600, facecolor='w', edgecolor='w',orientation='portrait',transparent=True,bbox_inches="tight" )
plt.show()


fig, ax = plt.subplots(figsize=(10, 5), sharex=True)
# plt.title("Test data vs predictions (kips)(Epoch "+str(epoch_es)+")(Site ID "+str(SITE_ID)+")("+str(BOUND[i])+")", fontsize=14)
plt.ylabel('ADT')
plt.xlabel('Year')
sns.lineplot(data=valid["y_test"], linewidth=1.0, linestyle = 'solid', color= 'black')
sns.lineplot(data=valid["y_pred"], linewidth=1.0, linestyle = 'dashed', color= 'black')
plt.legend(['Test Data', 'Test Prediction'], loc='best')
plt.grid(axis = 'x')
plt.xticks(rotation = 30)
#             plt.fill_between(valid.index, under_line, over_line, color='black', alpha=.1)
# plt.savefig(new_path +'\\' +new_folder +'\\'+str(BOUND[i])+'~Train data vs test (Site ID '+str(SITE_ID)+').png', dpi=600, facecolor='w', edgecolor='w',orientation='portrait',transparent=True,bbox_inches="tight" )
plt.show()


# summarize history for loss
plt.plot(history.history['loss'], linestyle = 'solid', color = 'gray')
plt.plot(history.history['val_loss'], linestyle = 'solid', color = 'Black')
# plt.title('Model Loss (Epoch '+str(epoch_es)+')(Site ID '+str(SITE_ID)+')('+str(BOUND[i])+')', fontsize=14)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Test'], loc='best')
# plt.savefig(new_path +'\\' +new_folder +'\\'+str(BOUND[i])+'~Model Loss vs Epoch Plot (Site ID '+str(SITE_ID)+').png', dpi=600, facecolor='w', edgecolor='w',orientation='portrait',transparent=True,bbox_inches="tight" )
plt.show()